# Internal Datastructure: Bus branch model, Admittance and Jacobian Matrix


This jupyter notebooks explains how to access and interpret the internal datastructure with relevant matrices.

### Internal Datastructure

We use the simple example network from the create_simple tutorial as an example for how to access internal calculation parameters:

<img src="pics/example_network_simple.png">


In [1]:
import pandapower as pp
import pandapower.networks as nw

net = nw.example_simple()
print(net)

This pandapower network includes the following parameter tables:
   - bus (7 elements)
   - load (1 element)
   - sgen (1 element)
   - gen (1 element)
   - switch (8 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (4 elements)
   - trafo (1 element)


First, we run a power flow in this network:

In [2]:
pp.runpp(net)

When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accesed after power flow:

In [3]:
net._ppc

{'baseMVA': 1,
 'version': 2,
 'bus': array([[ 0.00000000e+00,  3.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.02000000e+00,  0.00000000e+00,
          1.10000000e+02,  1.00000000e+00,  2.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  9.60000000e-01,
          1.00000000e+00,  1.02084170e+00,  3.21777771e-02,
          1.10000000e+02,  1.00000000e+00,  2.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 2.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.02505441e+00,  1.80254139e+00,
          2.00000000e+01,  1.00000000e+00,  2.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 3.00000000e+00,  2.00000000e+00,  0.00000000e+00,


For information on how this datastructure is defined, please referr to the MATPOWER documentation.

## Nodal Point Admittance Matrix

The nodal point admittance matrix is saved in the ppc and can be accessed directly:

In [4]:
net._ppc["internal"]["Ybus"].todense()

matrix([[ 2983.234714  -7157.02635809j, -2983.234714  +7159.76331361j,
             0.           +0.j        ,     0.           +0.j        ,
             0.           +0.j        ,     0.           +0.j        ],
        [-2983.234714  +7159.76331361j,  2990.35626947-7364.28068082j,
            -7.11455564 +208.20907269j,     0.           +0.j        ,
             0.           +0.j        ,     0.           +0.j        ],
        [    0.           +0.j        ,    -7.11455564 +208.20907269j,
          1608.40491554-1678.15899439j,  -889.60186671 +816.68368091j,
          -711.68149336 +653.34694473j,     0.           +0.j        ],
        [    0.           +0.j        ,     0.           +0.j        ,
          -889.60186671 +816.68368091j,  1027.81021222 -903.21268537j,
             0.           +0.j        ,  -138.20834551  +86.56929539j],
        [    0.           +0.j        ,     0.           +0.j        ,
          -711.68149336 +653.34694473j,     0.           +0.j        ,
  

Note that the nodal point admittance matrix is given in per unit values.

## Jacobian Matrix

The jacobian Matrix J in the last iteration step is also stored in the ppc and can be accessed:

In [5]:
net._ppc["internal"]["J"].todense()

matrix([[ 9.55699935e+02,  0.00000000e+00, -8.63856353e+02,
          0.00000000e+00, -9.18435819e+01,  0.00000000e+00,
         -9.14857404e+02,  0.00000000e+00, -1.42353628e+02],
        [ 0.00000000e+00,  7.67444779e+03, -2.17539786e+02,
          0.00000000e+00,  0.00000000e+00,  3.05268038e+03,
         -1.38257952e+01,  0.00000000e+00,  0.00000000e+00],
        [-8.60660279e+02, -2.17999782e+02,  1.76330297e+03,
         -6.84642909e+02,  0.00000000e+00, -6.95800979e-01,
          1.64870255e+03, -7.30767769e+02,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -6.87447705e+02,
          6.87447705e+02,  0.00000000e+00,  0.00000000e+00,
         -7.27777861e+02,  7.29816081e+02,  0.00000000e+00],
        [-9.18403982e+01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  9.18403982e+01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  1.42355564e+02],
        [ 0.00000000e+00, -3.11630342e+03,  1.41721924e+01,
          0.00000000e+00,  0.000000

The jacobian matrix is also given in per unit values.

## Mapping the Buses

The pandapower indices are not equal to the ppc indices for several reasons. Some buses are fused together in case of closed bus-bus switches and auxiliary buses are created for elements like extended wards or three winding transformers. There is however a mapping between pandapower indices and ppc indices that is created during the conversion to keep track of the dependencies that is also stored in the net:

In [6]:
net._pd2ppc_lookups["bus"]

array([0, 1, 1, 2, 2, 3, 4])

To get a ppc index from the pandapower index, simply call the lookup like this:

In [7]:
pandapower_bus_idx = 3
ppc_index = net._pd2ppc_lookups["bus"][pandapower_bus_idx]
print(ppc_index)

2


As you can see, pandapower bus index 3 corresponds to ppc bus index 2. So if we would like to find the diagonal entry of the Ybus matrix for bus 2, we could now access it with that internal index:

In [8]:
Ybus = net._ppc["internal"]["Ybus"]
int_idx = net._pd2ppc_lookups["bus"][ppc_index]
Ybus[int_idx, int_idx]

(2990.356269470474-7364.280680821408j)

We can also see that some buses are mapped to the same internal bus, such as bus 1 and bus 2:

In [9]:
print(net._pd2ppc_lookups["bus"][1])
print(net._pd2ppc_lookups["bus"][2])

1
1


That is because buses 1 and 2 are connected by a closed bus-bus switch and are therefore represented internally as the same bus:

In [10]:
net.switch.loc[0]

bus           1
element       2
et            b
type         CB
closed     True
name       None
Name: 0, dtype: object